# Context

This is what is investigated here

# Imports

In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

### Import our custom utility methods

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

import utils.levelset_flow
from utils.levelset_flow import *

The following modules are available

In [ ]:
print_module_methods(utils.diverse)

In [ ]:
print_module_methods(utils.visualization_tools)

In [ ]:
print_module_methods(utils.data_transformations)

In [ ]:
def motion_statistic(y_comp, x_comp, mask):
    out = []

    dirs = np.rad2deg(np.arctan2(y_comp,x_comp))
    mags = np.sqrt(x_comp**2, y_comp**2)

    for mag, dir in zip(mags,dirs):
        vals = []
        for low in range(-180,180,10):
            high = low + 10
            mask = np.logical_and(~mask,np.logical_and(dir > low, dir <= high))
            vals.append(np.sum(mag[mask]))
        out.append(vals)
        
    out = np.array(out).T        
    return out

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "source_data")

files = []
files.append(os.path.join(source_folder,"runstart16_X1.tif"))
files.append(os.path.join(source_folder,"runstart16_X2.tif"))

filepath = files[0]

In [ ]:
mask = np.array(Image.open(os.path.join(source_folder,"mask_runstart16_X_left.png")))==0
mask = mask [:,:,0]
mask = ~mask

In [ ]:
frames = np.array(skimage.io.imread(os.path.join(filepath)), dtype=np.double)[:1000]
mean = np.mean(frames,axis=0)#pixelwise mean
difference = framewise_difference(frames, mean, bigdata=True)
mean = None

In [ ]:
frames = difference.copy()
frames = remove_frequency_from_pixel_vectors(frames,15,20)

In [ ]:
frames = gaussian_filter(frames,2)

In [ ]:
expected = np.load("10_bins_expected_images_polynomial.npy")

# Remove the expected image from each frame

In [ ]:
#frames -= np.min(frames)

In [ ]:
frames = substract_expected(frames, expected, True)

In [ ]:
frames = normalize(frames)

In [ ]:
unmasked = frames.copy()

In [ ]:
frames = apply_mask(frames, mask)

# Check out unmasked data

In [ ]:
%%capture
ani = show_video((gaussian_filter(unmasked[560:560+200],1)), n_frames=100)

In [ ]:
HTML(ani)

# Correct vector fields

In [ ]:
n_levels = 50
ls = levelsets(frames[560:765], .1, 1.0, n_levels)
closed = framewise_closing(ls[:], n_iterations=12, smoothing=0, post_smoothing=0)#4
ls1 = closed.reshape(len(closed),n_levels, closed[0].shape[0]//n_levels,closed[0].shape[1])
reassembled = normalize(np.mean(ls1[:,:,:,:],axis=(1)))
smooth_reassembled = gaussian_filter(reassembled,[0,10,10])
y_comp_expected, x_comp_expected = expected_flow(smooth_reassembled, 202)

In [ ]:
x_comp, y_comp = horn_schunck(gaussian_filter(reassembled,[0,10,10]), 202)

In [ ]:
y_comp_corrected, x_comp_corrected = post_process_vector_fields(y_comp, y_comp_expected, x_comp, x_comp_expected, mask, logscale = True)

In [ ]:
#%%capture
#ani = show_video(normalize(unmasked[560:560+202]), n_frames=200)

In [ ]:
def zero_crossings(vector):
    """ Returns the positions of the zero crossings
    Args:
        vector: Vector with potentially positive and negative values
    Returns:
        zero_crossings: Position of the zero crossings
    """
    return np.where(np.diff(np.sign(vector)))[0]

In [ ]:
res = motion_statistic(np.array(y_comp_corrected), np.array(x_comp_corrected), mask)
res = np.log(np.abs(res)+1)

fig, ax = plt.subplots(1, figsize=(15,15))
y_ticks = np.arange(5)*9
y_ticks[-1] -=1
ax.set_yticks(y_ticks)
ax.set_yticklabels(["left","up","right","down","left"])

ax.imshow(np.vstack(res))

In [ ]:
#center of mass
cm_left = np.array([center_of_mass(f) for f in unmasked[560:560+202,25:240,50:160]]).T
cm_left_y = normalize(cm_left[0])
cm_left_x = normalize(cm_left[1])

upwards_trend = np.array(y_comp_corrected).copy()
upwards_trend[upwards_trend<0] = np.nan
downwards_trend = np.array(y_comp_corrected).copy()
downwards_trend[upwards_trend>0] = np.nan
downwards_trend *= -1
upwards_trend *= -1
upwards_trend = np.nanmean(upwards_trend, axis=(1,2))
downwards_trend = np.nanmean(downwards_trend, axis=(1,2))


leftwards_trend = np.array(x_comp_corrected).copy()
leftwards_trend[leftwards_trend<0] = np.nan
rightwards_trend = np.array(y_comp_corrected).copy()
rightwards_trend[rightwards_trend>0] = np.nan
leftwards_trend = np.nanmean(leftwards_trend, axis=(1,2))
rightwards_trend = np.nanmean(rightwards_trend, axis=(1,2))

dist = 4
upwards_trend1 = np.mean(res[9-dist:9+dist,:],axis=0)
downwards_trend1 = -np.mean(res[28-dist:27+dist,:],axis=0)

In [ ]:
fig, ax = plt.subplots(7, figsize=(12,10))
fig.tight_layout()

# Framewise mean
ax[0].set_title("df/dt")
ax[0].plot(normalize(np.mean(reassembled[:202], axis=(1,2))), label="Corrected for blood vessels")
ax[0].plot(normalize(np.mean(frames[560:560+202], axis=(1,2))), label="Contrast to expected")
ax[0].plot(normalize(np.mean(difference[560:560+202], axis=(1,2))), label="Contrast to pixelwise mean")
ax[0].legend(loc="right")

ax[0].plot()

ax[0].set_xlabel("Frame index")
ax[0].set_ylabel("Normal frame-mean")

# EMPTY AXIS
ax[1].axis('off')

# DIRECTION OF SPREDING MOTION (Up down)
ax[2].set_title("Direction of spreading motion")
ax[2].axhline(0, c="gray")

ax[2].plot(downwards_trend, label="Sum of Y-component of upwards vectors")
ax[2].plot(upwards_trend, label="Sum of Y-component of downwards vectors")
#ax[2].plot(upwards_trend1, label="Upwards +- 30 degrees")
#ax[2].plot(downwards_trend1, label="Downwards +- 30 degrees" )
ax[2].legend(loc="lower right")

#ax[2].plot(upwards_trend/(.01+normalize(np.mean(frames[560:560+202], axis=(1,2)))))
ax[2].set_xlabel("Frame index")
ax[2].set_ylabel("Motion [up/down]")

# DIRECTION OF SPREDING MOTION (Left right)
ax[3].axhline(0, c="gray")
ax[3].set_xlabel("Frame index")
ax[3].set_ylabel("Motion [left/right]", labelpad=11.5)
ax[3].plot(leftwards_trend, label="Sum of X-component of leftwards vectors")
ax[3].plot(rightwards_trend, label="Sum of X-component of rightwards vectors")
ax[3].legend(loc="upper right")


# EMPTY AXIS
ax[4].axis('off')


ax[5].set_title("Center of gravity of pixels")

ax[5].plot(cm_left_y)
ax[5].set_ylabel("Y-component")
ax[5].set_xlabel("Frame index")

ax[6].plot(cm_left_x)
ax[6].set_ylabel("X-component")
ax[6].set_xlabel("Frame index")
plt.subplots_adjust(hspace=.0)

In [ ]:
y_comp_corrected, x_comp_corrected = post_process_vector_fields(y_comp, y_comp_expected, x_comp, x_comp_expected, mask, logscale = True, pre_log_factor = 3000)

In [ ]:
%%capture
i = 0
fig, ax = display_combined(x_comp_corrected[0],y_comp_corrected[0], reassembled[1], quivstep = 10)
time_text = plt.figtext(0.5, 0.01, "time " + str(0), ha="center", fontsize=18)

def animate(i):
    global time_text
    print(".", end ="")    
    display_combined(y_comp_corrected[i],x_comp_corrected[i],reassembled[i], fig=fig, ax=ax, scale=1, quivstep = 5, head_width=2.5)
    time_text.set_text('time = %.1d' % i)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=200)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
difference = gaussian_filter(difference, 2)

In [ ]:
%%capture
i = 0
fig, ax = plt.subplots(4,1, figsize=(12,12), gridspec_kw={'height_ratios': [1,.3,.3,.3]})
time_text = plt.figtext(0.5, 0.01, "time " + str(0), ha="center", fontsize=18)
ax[1].axis('off')
ax[2].plot(upwards_trend)
ax[2].plot(downwards_trend)
ax[3].plot(leftwards_trend)
ax[3].plot(rightwards_trend)


plt.subplots_adjust(hspace=.0)


line = ax[2].axvline(i, c="red")
line1 = ax[3].axvline(i, c="red")


def animate(i):
    global time_text
    print(".", end ="")
    ax[0].imshow(difference[560+i])
    line.set_data([i, i], [0, 1])
    line1.set_data([i, i], [0, 1])

    time_text.set_text('time = %.1d' % i)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=200)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
def histograms_of_motion(x_comp, y_comp, bins = 36, weighted = True):
    x_comp = x_comp.reshape(x_comp.shape[0],x_comp.shape[1]*x_comp.shape[2])
    y_comp = y_comp.reshape(y_comp.shape[0],y_comp.shape[1]*y_comp.shape[2])
    
    histograms = []
    
    for x, y in zip(x_comp, y_comp):
        x = x[~np.isnan(x)]
        y = y[~np.isnan(y)]
        
        mask = ~np.logical_and(x==0,y==0)
        x = x[mask]
        y = y[mask]
        assert len(x) == len(y)
        mag = np.sqrt(x**2, y**2)
        dir = np.rad2deg(np.arctan2(y,x))
        if weighted:
            hist = np.histogram(dir, weights = mag, bins=bins, range=(-180,180))[0]
        else:
            hist = np.histogram(dir, bins=bins, range=(0,360))[0]
        #if np.sum(hist) !=0:
        #    histograms.append(normalize(hist))
        #else:
        histograms.append(hist)
    return np.array(histograms).T

In [ ]:
hist = histograms_of_motion(np.array(x_comp), np.array(y_comp))

In [ ]:
plt.imshow(mask)

In [ ]:
y_comp_corrected, x_comp_corrected = post_process_vector_fields(y_comp, y_comp_expected, x_comp, x_comp_expected, mask, logscale = False)

In [ ]:
res.shape

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], reassembled[1], quivstep = 10)
time_text = plt.figtext(0.5, 0.01, "time " + str(0), ha="center", fontsize=18)

def animate(i):
    global time_text
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], reassembled[i], fig=fig, ax=ax, scale=500, quivstep = 5)
    time_text.set_text('time = %.1d' % i)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=200)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
from pynhhd import nHHD

def helmholtz_decomposition(y_comp, x_comp):
    vfield = np.array([y_comp,x_comp])
    vfield = np.einsum("ijk->kij",vfield)
    vfield = np.einsum("ijk->kij",vfield)
    dims = (vfield.shape[0],vfield.shape[1])
    nhhd = nHHD(grid=dims, spacings=(0.1,0.1))
    nhhd.decompose(vfield)
    return nhhd.r, nhhd.d, nhhd.h

In [ ]:
y_comp[np.isnan(y_comp)] = 0

In [ ]:
x_comp[np.isnan(x_comp)] = 0

In [ ]:
r, d, h = helmholtz_decomposition(y_comp[10], x_comp[10])

In [ ]:
display_combined(r[:,:,1],r[:,:,0],tensor_of_interest[10],quivstep=3,scale=100, head_width = 2)
plt.show()

In [ ]:
i_frame = 10
display_combined(x_comp[i_frame],y_comp[i_frame],np.sqrt(y_comp[3]**2+x_comp[3]**2)*100, quivstep=3)

In [ ]:
%%capture
lengths = np.sqrt(x_comp**2+y_comp**2)
ani = show_video(lengths*10, n_frames=90)

In [ ]:
%%capture
ani = show_video(normalize(max_filtered[550:]), normalize(difference[550:]), n_frames=50, vmin = .1, vmax=.3)

In [ ]:
HTML(ani)

In [ ]:
%%capture
ani = show_video(normalize(gaussian_filter(interpolated,2)), n_frames=199, vmin = .1, vmax=.5)

In [ ]:
HTML(ani)

In [ ]:
x_comp, y_comp = horn_schunck(normalize(gaussian_filter(interpolated,2)), 150)

In [ ]:
np.save("test_vector_field.npy",np.array([x_comp[75],y_comp[75]]))

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], interpolated[1])
start = 70
n_frames = 10

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], interpolated[i+1], fig=fig, ax=ax, scale=1000)

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=n_frames)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
#from scipy.ndimage.measurements import center_of_mass

In [ ]:
left_hemisphere = difference[:,:,:160]
right_hemisphere = difference[:,:,160:]
cm_left = np.array([center_of_mass(f) for f in left_hemisphere])
cm_right = np.array([center_of_mass(f) for f in right_hemisphere])

fig, ax = plt.subplots(3)
fig.suptitle("Center of mass (contrast to mean image)")
ax[0].plot(np.sum(difference, axis=(1,2)))

ax[1].plot(cm_right[:,0])
ax[1].plot(cm_left[:,0])

ax[2].plot(cm_right[:,1])
ax[2].plot(cm_left[:,1])

In [ ]:
left_hemisphere = frames[:,:,:160]
right_hemisphere = frames[:,:,160:]
cm_left = np.array([center_of_mass(f) for f in left_hemisphere])
cm_right = np.array([center_of_mass(f) for f in right_hemisphere])

fig, ax = plt.subplots(3)
fig.suptitle("Center of mass (contrast expected image)")

ax[0].plot(np.sum(difference, axis=(1,2)))

ax[1].plot(cm_right[:,0]-np.mean(cm_right[:,0]))
ax[1].plot(cm_left[:,0]-np.mean(cm_left[:,0]))

ax[2].plot(cm_right[:,1])
ax[2].plot(cm_left[:,1])

In [ ]:
%%capture
ani = show_video(normalize(max_filtered[600:]), normalize(difference[600:]), n_frames=200, vmin = .3, vmax=.7)

In [ ]:
# use median!